Importar Librerias

In [2]:
import pandas as pd
import numpy as np

Cargar el dataset


In [3]:
file_path = '3._Accidentes_de_Transito_ocurridos_en_el_Municipio_de_Bucaramanga_20250430.csv'
df = pd.read_csv(file_path)
print("Cargado:"); df.info()      # Ver qué tipos y cuántos nulos hay al inicio
df.head()

Cargado:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39193 entries, 0 to 39192
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ORDEN                    39193 non-null  int64 
 1   FECHA                    39193 non-null  object
 2   AÑO                      39193 non-null  int64 
 3   MES                      39193 non-null  object
 4   DÍA                      39193 non-null  object
 5   GRAVEDAD                 39193 non-null  object
 6   PEATON                   39193 non-null  int64 
 7   AUTOMOVIL                39193 non-null  int64 
 8   CAMPERO                  39193 non-null  int64 
 9   CAMIONETA                39193 non-null  int64 
 10  MICRO                    39193 non-null  int64 
 11  BUSETA                   39193 non-null  int64 
 12  BUS                      39193 non-null  int64 
 13  CAMION                   39193 non-null  int64 
 14  VOLQUETA                 3919

,ORDEN,FECHA,AÑO,MES,DÍA,GRAVEDAD,PEATON,AUTOMOVIL,CAMPERO,CAMIONETA,...,VOLQUETA,MOTO,BICICLETA,OTRO,BARRIO,HORA,ENTIDAD,COMUNA,Propietario de Vehículo,DIURNIO/NOCTURNO
0,1,2012-01-01T00:00:00.000,2012,01. Enero,07. Domingo,Con heridos,0,1,0,0,...,0,0,0,0,Mutis,1899-12-31T12:15:00.000,AGENTES DTB,17. MUTIS,Particular,Diurno
1,2,2012-01-01T00:00:00.000,2012,01. Enero,07. Domingo,Solo daños,0,1,0,1,...,0,0,0,0,Regaderos Norte,1899-12-31T14:00:00.000,AGENTES DTB,02. NORORIENTAL,Empresa,Diurno
2,3,2012-01-01T00:00:00.000,2012,01. Enero,07. Domingo,Solo daños,0,0,0,1,...,0,0,0,0,Cabecera Del Llano,1899-12-31T12:00:00.000,AGENTES DTB,12. CABECERA DEL LLANO,Particular,Diurno
3,4,2012-01-01T00:00:00.000,2012,01. Enero,07. Domingo,Solo daños,0,1,0,1,...,0,0,0,0,Norte Bajo,1899-12-31T18:30:00.000,AGENTES DTB,03. SAN FRANCISCO,Empresa,Nocturno
4,5,2012-01-01T00:00:00.000,2012,01. Enero,07. Domingo,Con heridos,1,0,0,0,...,0,1,0,0,Dangond,1899-12-31T00:30:00.000,AGENTES DTB,11. SUR,Particular,Nocturno


Corregir errores de codificación

In [4]:
df['GRAVEDAD'] = df['GRAVEDAD'] \
    .astype(str) \
    .str.replace('Ã±', 'ñ', regex=False) \
    .str.replace('daÃ±os', 'daños', regex=False)
print("\n Codificación corregida en GRAVEDAD:"); df['GRAVEDAD'].unique()


 Codificación corregida en GRAVEDAD:


array(['Con heridos', 'Solo daños', 'Con muertos', 'Con Muertos'],
      dtype=object)

Transformar FECHA y HORA a datetime

In [5]:
df['FECHA'] = pd.to_datetime(df['FECHA'], errors='coerce')
# HORA → primero datetime, luego extraer time
df['HORA'] = pd.to_datetime(df['HORA'], errors='coerce').dt.time
print("\n Tipos tras conversión:"); print(df[['FECHA','HORA']].dtypes)
print("Nulos en HORA:", df['HORA'].isna().sum())


 Tipos tras conversión:
FECHA    datetime64[ns]
HORA             object
dtype: object
Nulos en HORA: 3


Estandarizar variables categóricas

In [6]:
cat_cols = ['MES', 'DÍA', 'GRAVEDAD', 'BARRIO',
            'ENTIDAD', 'COMUNA', 'Propietario de Vehículo', 'DIURNIO/NOCTURNO']
for col in cat_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.strip()            # quita espacios al inicio/final
        .str.lower()            # todo en minúscula
        .str.replace(' +', ' ', regex=True)  # espacios múltiples → uno
        .str.title()            # capitaliza cada palabra
    )
print("\n Primeras categorías limpias:")
for col in cat_cols:
    print(f" - {col}: {df[col].unique()[:5]}")


 Primeras categorías limpias:
 - MES: ['01. Enero' '02. Febrero' '03. Marzo' '04. Abril' '05. Mayo']
 - DÍA: ['07. Domingo' '01. Lunes' '02. Martes' '03. Miercoles' '04. Jueves']
 - GRAVEDAD: ['Con Heridos' 'Solo Daños' 'Con Muertos']
 - BARRIO: ['Mutis' 'Regaderos Norte' 'Cabecera Del Llano' 'Norte Bajo' 'Dangond']
 - ENTIDAD: ['Agentes Dtb' 'Agentes Policia Nacional']
 - COMUNA: ['17. Mutis' '02. Nororiental' '12. Cabecera Del Llano'
 '03. San Francisco' '11. Sur']
 - Propietario de Vehículo: ['Particular' 'Empresa']
 - DIURNIO/NOCTURNO: ['Diurno' 'Nocturno']


Manejo de valores nulos en HORA

In [7]:
df['HORA'] = df['HORA'].fillna(pd.to_datetime('00:00').time())
print("\n Nulos en HORA tras relleno:", df['HORA'].isna().sum())


 Nulos en HORA tras relleno: 0


Renombrar columnas para legibilidad

In [8]:
df.columns = (
    df.columns
      .str.strip()
      .str.replace(' ', '_')
      .str.replace('/', '_')
      .str.normalize('NFKD')  # elimina tildes si quisieras
      .str.encode('ascii', errors='ignore')
      .str.decode('utf-8')
      .str.upper()
)
print("\n Columnas renombradas:"); print(df.columns.tolist())


 Columnas renombradas:
['ORDEN', 'FECHA', 'ANO', 'MES', 'DIA', 'GRAVEDAD', 'PEATON', 'AUTOMOVIL', 'CAMPERO', 'CAMIONETA', 'MICRO', 'BUSETA', 'BUS', 'CAMION', 'VOLQUETA', 'MOTO', 'BICICLETA', 'OTRO', 'BARRIO', 'HORA', 'ENTIDAD', 'COMUNA', 'PROPIETARIO_DE_VEHICULO', 'DIURNIO_NOCTURNO']


Verificación final

In [10]:
print("\n--- Info final del DataFrame limpio ---")
df.info()
df.head()

# =========================
# 9. Exportar a CSV limpio
# =========================
output_path = 'accidentes_bucaramanga_2012_limpio.csv'
df.to_csv(output_path, index=False)
print(f"\n Exportado a {output_path}")


--- Info final del DataFrame limpio ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39193 entries, 0 to 39192
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   ORDEN                    39193 non-null  int64         
 1   FECHA                    39193 non-null  datetime64[ns]
 2   ANO                      39193 non-null  int64         
 3   MES                      39193 non-null  object        
 4   DIA                      39193 non-null  object        
 5   GRAVEDAD                 39193 non-null  object        
 6   PEATON                   39193 non-null  int64         
 7   AUTOMOVIL                39193 non-null  int64         
 8   CAMPERO                  39193 non-null  int64         
 9   CAMIONETA                39193 non-null  int64         
 10  MICRO                    39193 non-null  int64         
 11  BUSETA                   39193 non-null  int64      